In [1]:
import sys
sys.path.insert(0, "../../code")

import networkx as nx
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import ariadne as a

from datasets import *
from api import *

In [7]:
100_000

100000

In [2]:
print(sdl)


    type Query {
        staticData: StaticData!
        # routes(payment: Payment): [ Route! ]!
        # stats: Stats!
    }

    type StaticData {
        origins: [ Party! ]! # 10-20 options
        destinations: [ Destination! ]! # 100-10000 options
        assetCategories: [ String! ]!
        currency: String!
        amount: Float!
    }

    type Party {
         bic: String! # 8chars typically
         name: String!
         countryCode: String!
         countryName: String!
         city: String!
    }

    type Destination {
        party: Party!
        accounts: [ String! ]!
    }

    type Payment {
        originBic: String!
        destinationBic: String!
        assetCategory: String!
        currency: String!
        amount: Float!
    }

    type Hop {
         source: Party!
         target: Party!
         payment: Payment!
         fxRate: Float # optional
         timeTakenMinutes: Int!
         crossBorder: Boolean!
    }

    type Route {
        originalPaym